# PROJECT GEGEVENSBANKEN 2018 - DEEL 3: QUERIES

## 1. Inleiding

Cf. het document `prerequisites.pdf` in de `docs` folder. 

### 1.1 Software

In eerste instantie moeten de correcte packages geladen worden. We raden **sterk** aan om je ook tot deze packages te beperken, gezien wij de ingevulde methodes zullen runnen op onze eigen machines. 

Zelfde opmerking voor de gebruikte python versie, wij veronderstellen 3.6.

In [ ]:
# Benodigde packages

import json            # Package om .json files in te laden (bvb kolomnamen zijn zo opgeslagen)
import getpass         # Package om een paswoordveldje te genereren.
import mysql.connector # MySQL package
import numpy as np
import os
import pandas as pd    # Populaire package voor data-verwerking
import sys

In [ ]:
sys.version_info       # Check python versie, wij veronderstellen 3.6

### 1.2 Interageren met een gegevensbank

In deze sectie geven we al drie methodes mee om jullie leven al wat makkelijker te maken

- `verbind met GB`
    - Dit is functie die toelaat om gemakkelijk verbinding te maken met een database. De `username`, `hostname` en `gegevensbanknaam` worden als argumenten meegegeven met de functie. Deze functie geeft een `connection` object als output. Zo'n object is wat de `connect()` methode van het `mysql.connector` package teruggeeft. Dit is conform de python DB-API (cf. https://www.python.org/dev/peps/pep-0249/), een soort standaard waaraan python DB packages voldoen. 
    
    Moest deze functie niet correct functioneren op je eigen besturingssysteem: voel je vrij om een variant te schrijven. Bij de correctie gebruiken wij onze eigen implementatie om een mysql `connection` object te maken. Deze functie is dus enkel voor jezelf van belang.
    

- `run_query`  
    - Deze functie runt een gegeven `query` op een gegeven `connectie`, en geeft het resultaat terug.
    
    
- `res_to_df`
    - Deze functie giet het resultaat van een query (`query_result`) in een pandas dataframe, met vooraf gespecifieerde kolomnamen (`column_names`).

In [ ]:
def verbind_met_GB(username, hostname, gegevensbanknaam):
    """
    Maak verbinding met een externe gegevensbank
    
    :param  username:          username van de gebruiker, string
    :param  hostname:          naam van de host, string.
                               Dit is in het geval van een lokale server gewoon 'localhost'
    :param  gegevensbanknaam:  naam van de gegevensbank, string.
    :return connection:        connection object, dit is wat teruggeven wordt 
                               door connect() methods van packages die voldoen aan de DB-API
    """
    
    password = getpass.getpass() # Genereer vakje voor wachtwoord in te geven
    
    connection = mysql.connector.connect(host=hostname,
                                         user=username,
                                         passwd=password,
                                         db=gegevensbanknaam)
    return connection


def run_query(connection, query):
    """
    Voer een query uit op een reeds gemaakte connectie, geeft het resultaat van de query terug
    """
    
    # Making a cursor and executing the query
    cursor = connection.cursor()
    cursor.execute(query)
    
    # Collecting the result and casting it in a pd.DataFrame
    res = cursor.fetchall()
    
    return res


def res_to_df(query_result, column_names):
    """
    Giet het resultaat van een uitgevoerde query in een 'pandas dataframe'
    met vooraf gespecifieerde kolomnamen.
    
    Let op: Het resultaat van de query moet dus exact evenveel kolommen bevatten
    als kolomnamen die je meegeeft. Als dit niet het geval is, is dit een indicatie
    dat je oplossing fout is. (Gezien wij de kolomnamen van de oplossing al cadeau doen)
    
    """
    df = pd.DataFrame(query_result, columns=column_names)
    return df

### 1.3 Kolomnamen en input parameters

We leggen op voorhand reeds de **kolomnamen van de oplossingen**, en de **naam en types van de inputparameters** vast. Hier moet je je dus aan houden en mag je dus niks aan wijzigen.

- Het aantal kolommen (en hun volgorde) van jullie oplossing en de onze moeten exact overeen komen, vandaar dat de kolomnamen hieronder gegeven zijn. Dit komt trouwens van pas bij het opstellen van je queries! 
    
    
- Dankzij de voorbeeldparameters, die al gegeven zijn in de functiedefinitie e.g., 
        query_42(connection, col_names, super_voorbeeldparam = 101)
  weten jullie exact welke vorm en type (integer, lijst, etc) de inputparameters zullen hebben. Wijzig zeker niets aan de naam van die parameters (i.e.,*super_voorbeeldparam* blijft *super_voorbeeldparam*). De default waarden die we ingevuld hebben zijn ter illustratie, zorg ervoor dat je query ook met andere waarden dan de defaults werkt!

**Oplossingen die deze vorm niet respecteren, zullen crashen op onze machines en resulteren in een score van 0 op die query.**

In [ ]:
# Dictionary van kolomnamen inladen

filename = os.path.join(os.getcwd(), 'solution', 'all_q_colnam.json')
col_names = json.load(open(filename, 'r'))    

col_names # Inspecteer dictionary

## 2. Hoe deze notebook in te vullen

Hieronder volgen 10 onvolledige methodes/functies (e.g., `query_42(c, col_names, super_voorbeeldparam = ['joske', 'jef'])`). Aan jullie om ze aan te vullen zodat:
    1. Een correcte query gegenereerd wordt
    2. De query uitgevoerd wordt
    3. Het resultaat wordt teruggegeven in een Pandas DataFrame

Voor stap 2 en 3 zijn de nodige methodes al voorhanden, i.e.: `run_query(connection, query)` en `res_to_df(res, column_names)`. Jullie werk zal dus vooral bestaan uit stap 1, queries genereren.
  
    
    
Elke functie heeft minstens 2 inputargumenten:
    1. `connection`:   Een connection object 
    2. `column_names`: De kolomnamen van het Pandas DataFrame
    
Gevolgd door eventuele extra argumenten (e.g., `super_voorbeeldparam = ['joske','jef']`) van de inputparameters. 

**Nogmaals: verander niets aan de namen van de methodes, namen van de parameters en de kolomnamen van de resulterende DataFrames. Wijzigingen hieraan leiden onvermijdelijk tot een score van 0 op die query.**

Je kan naar believen extra cellen toevoegen om je queries te testen, resultaten te inspecteren etc. We vragen jullie om de finale, ingevulde methodes te kopiëren naar een extern script dat enkel en alleen deze oplossingen bevat. Cf. de laatste sectie voor instructies omtrent het effectieve indienen.

## 3. Voorbeeld

Om jullie al wat op weg te zetten volgt hier een voorbeeldje over hoe je te werk kan gaan.

### 3.1 Query EXAMPLE

**Beschrijving**

Het resultaat van deze functie is een Pandas DataFrame met teamnaam, jaar en aantal homeruns van teams die meer dan een gegeven aantal *`homeruns`* hadden in dat jaar.

In [ ]:
def query_EX(connection, column_names, homeruns=20):
    # Bouw je query
    query="""
    select t.name, t.yearID, t.HR
    from Teams as t
    where t.HR > {};
    """.format(homeruns) # TIP: Zo krijg je parameters in de string (samen met `{}` in de string)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

Eerst maken we een verbindingsobject met de databank

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

Vervolgens runnen we onze query, en inspecteren we het resultaat.

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_01'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_EX(c, col_names['query_01'], homeruns=10)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

En we runnen onze query nogmaals met een andere waarde voor de parameter `jaar`

In [ ]:
# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_EX(c, col_names['query_01'], homeruns=40)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

## 4. Queries

### 4.1 Query 01

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe met: de teamnaam, het jaar, en het aantal homeruns per team, en dit voor alle teams, gesorteerd op aantal homeruns van hoog naar laag.

In [ ]:
def query_01(connection, column_names):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.2 Query 02

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe met: de voornaam, achternaam, geboortejaar, geboortemaand, geboortedag van spelers die hun eerste major league appearance maakten na een gegeven *`datum`*. 

De tabel is oplopend alfabetisch gesorteerd op achternaam.

In [ ]:
def query_02(connection, column_names, datum = '1980-01-16'):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.3 Query 03

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat per club: de clubnaam en de voor- en achternaam van alle managers weergeeft, die ooit voor de club gewerkt hebben als niet-playermanager. Per club mag een welbepaalde manager slechts 1 keer in het resultaat voorkomen. 

Sorteer oplopend alfabetisch op clubnaam.

In [ ]:
def query_03(connection, column_names):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.4 Query 04

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe met gegevens van teams (teamnaam, rang, aantal wins en losses) en van managers (voor- en achternaam) zodanig dat
1. de desbetreffende manager is opgenomen in de hall of fame na *`datum x`* 
2. de manager in kwestie was ooit, na *`datum y`*, manager van het team in kwestie

De tabel moet gesorteerd zijn op teamnaam en rang (alfabetisch oplopend).

In [ ]:
def query_04(connection, column_names, datum_x='1980-01-01', datum_y='1980-01-01'):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.5 Query 05

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat de naam van de teams bevat die na 1980 minstens 1 manager hebben gehad die speler-manager was. 

De tabel moet oplopend gesorteerd worden op teamnaam. De tabel mag geen dubbels bevatten.

In [ ]:
def query_05(connection, column_names):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.6 Query 06

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat bestaat uit de teamnaam, rang, jaar, aantal wins en losses van de teams waarvan dat jaar alle spelers die bij dat team speelden, meer verdienden bij dat team dan *`salaris`*.

Sorteer de gegevens volgens oplopend aantal wins.

In [ ]:
def query_06(connection, column_names, salaris=20000):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.7 Query 07

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat de achternaam en voornaam bevat van alle managers die tijdens hun carriere alle awards hebben gewonnen.

Sorteer oplopend alfabetisch op achternaam.

In [ ]:
def query_07(connection, column_names):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.8 Query 08

**Beschrijving**

Het resultaat van deze functie is een Pandas DataFrame dat voor een gegeven jaar een aantal statistieken van alle staten bevat waarbij de gemiddelde lengte van alle spelers geboren in die staat en opgenomen in de hall of fame na *`jaar`* groter is dan *`lengte`*. 

Voor die staten moet de tabel de volgende statistieken bevatten: het gemiddelde gewicht, de gemiddelde lengte, het gemiddeld aantal batting homeruns, en het gemiddeld aantal pitching saves van alle spelers (geboren in die staat) die in de hall of fame zijn opgenomen na *`jaar`*.

Sorteer oplopend alfabetisch op staat.

Nb. Lengte is uitgedrukt in inches.

In [ ]:
def query_08(connection, column_names, jaar=1990, lengte=75):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.9 Query 09

**Beschrijving**

Het resultaat van deze functie geeft een Pandas dataframe terug dat bestaat uit het jaar, de naam en het aantal homeruns uit de Teams tabel van het team met het 2e meeste aantal homeruns in *`jaar`*.

In [ ]:
def query_09(connection, column_names, jaar=1975):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### 4.10 Query 10

**Beschrijving**

Het resultaat van deze functie is een Pandas DataFrame dat voor een gegeven *`jaar`*; teamnaam, rang, en aantal gespeelde games van de teams weergeeft die in dat *`jaar`* ten minste 1 speler hebben die datzelfde *`jaar`* exact 1 trofee won.

Sorteer op teamnaam en rang.

In [ ]:
def query_10(connection, column_names, jaar=1990):
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

## 5. Hoe en wat in te dienen?

Nu je alle queries ingevuld en getest hebt, ben je klaar om je taak in te dienen.

- Maak een leeg bestand aan, en geef het de bestandsnaam met formaat: `dd_X_groep_YY.py`.
    - `dd` verwijst naar de dag van je oefenzitting, e.g. `wo` voor woensdag
    - `X`  is een integer die verwijst naar de volgnummer van je oefenzitting op die dag, e.g. `1`
    - `YY` zijn twee integers die verwijzen naar de volgnummer van je groepje, e.g.: `03`
    -  Een goede bestandsnaam is dus bijvoorbeeld: `wo_1_groep_03.py`


- Kopieer alle **INGEVULDE FUNCTIES EN NIETS ANDERS** naar dit script. Het script bevat dus enkel en alleen de methodes:
    - query_01(connection, column_names)
    - query_02(connection, column_names, datum = '1980-01-16')
    - etc, etc


- Voor de eerste 3 queries kan je dan je oplossing al eens testen via de verification notebook!
    - Eerst wordt je script automatisch gerund met verschillende parameters
    - De (eventuele) resultaten worden opgeslagen in csv files (in de `out` folder)
    - Die csv files worden vergeleken met de csv files van de oplossing (te vinden in de `solution` folder).
    - Elke query krijgt een score toegekend. Cf. https://en.wikipedia.org/wiki/F1_score.  
    - Een kort rapport wordt weergegeven die je pointers kan geven over wat er mis is met je query. 
        - TP: True Positives
        - TN: True Negatives
        - FP: False Positives
        

- Als je oplossing definitief is, submit je je `dd_X_groep_YY.py` via Toledo.